In [ ]:
from dotenv import load_dotenv
from pyprojroot import here
import os

load_dotenv()

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=5)
search_tool.invoke("When will the US Government shut down end?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from operator import itemgetter

from langchain_community.agent_toolkits import SparkSQLToolkit, create_spark_sql_agent
from langchain_community.utilities.spark_sql import SparkSQL

from databricks.connect import DatabricksSession

In [ ]:
spark_sql_llm = ChatGroq(model="deepseek-r1-distill-llama-70b", temperature=0)

agent_llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

agent_suffix = """Given the following user question, corresponding Spark SQL query, and Spark SQL result, answer the user question.\n
Question: {input}
Thought: I should look at the tables in the database to see what I can query.
{agent_scratchpad}"""

spark = DatabricksSession.builder.getOrCreate()
spark_sql = SparkSQL(spark_session=spark)

spark_sql_toolkit = SparkSQLToolkit(db=spark_sql, llm=spark_sql_llm)

In [ ]:
answer_prompt = PromptTemplate.from_template(system_role)

spark_sql_agent = create_spark_sql_agent(llm=, toolkit=spark_sql_toolkit, verbose=True, agent_executor_kwargs={"handle_parsing_errors": True})

# Test the chain
message = "How many schemas do I have in the company catalog? and what are their names?"
response = chain.invoke({"question": message})

In [ ]:
@tool
def query_spark_sql(query):
    """Query the Swiss Airline SQL Database and access all the company's information. Input should be a search query."""
    response = chain.invoke({"question": query})
    return response